In [1]:
import os
import pandas as pd
import nltk
import random
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df_data = pd.DataFrame()
for country in ["da", "sv", "no"]:
    folder_path = os.path.join("app", "lang_model", "data", country)
    files = list(filter(lambda x: x.endswith(".txt"), os.listdir(folder_path)))
    sentences = []
    for filename in files:
        single_file = os.path.join(folder_path, filename)
        with open(single_file, "r") as file:
            text = file.read().replace("\n", "")
            text = re.sub(r'\d','',text)
            sentences.extend(nltk.sent_tokenize(text))

    df_country = pd.DataFrame({"data": sentences})
    df_country["label"] = country
    df_data = pd.concat([df_data, df_country])

df_data

,data,label
0,Tanzania er administrativt inddelt i regioner...,da
1,På næste niveau er landet inddelt i distrikte...,da
2,Landsdel Midtjylland-Nordjylland er i forbinde...,da
3,"Den omfatter Østjyllands Storkreds, Nordjyllan...",da
4,Ved næste folketingsvalg vil den fordeling vær...,da
...,...,...
28700,De største lavvoene har flere aluminiumsstenge...,no
28701,De største lavvoene kan romme mer enn personer.,no
28702,I dag er det også vanlig å bruke ovn til fyrin...,no
28703,Falklandsøyenes politiske system er de styrend...,no


In [3]:
df_data.reset_index(drop=True, inplace=True)
df_data["data"] = df_data["data"].str.lower()
df_data

,data,label
0,tanzania er administrativt inddelt i regioner...,da
1,på næste niveau er landet inddelt i distrikte...,da
2,landsdel midtjylland-nordjylland er i forbinde...,da
3,"den omfatter østjyllands storkreds, nordjyllan...",da
4,ved næste folketingsvalg vil den fordeling vær...,da
...,...,...
81151,de største lavvoene har flere aluminiumsstenge...,no
81152,de største lavvoene kan romme mer enn personer.,no
81153,i dag er det også vanlig å bruke ovn til fyrin...,no
81154,falklandsøyenes politiske system er de styrend...,no


In [4]:
uniq = df_data["data"].unique()
size = len(uniq)
random.shuffle(uniq)
split = int(0.8 * size)
train = df_data[df_data["data"].isin(uniq[:split])]
test = df_data[df_data["data"].isin(uniq[split:])]

In [5]:
train.to_hdf("app/lang_model/data/train.hdf5", "data", mode="w")
test.to_hdf("app/lang_model/data/test.hdf5", "data", mode="w")

In [6]:
len(train), len(test)

(64966, 16190)

In [7]:
test.head(20)

,data,label
1,på næste niveau er landet inddelt i distrikte...,da
2,landsdel midtjylland-nordjylland er i forbinde...,da
3,"den omfatter østjyllands storkreds, nordjyllan...",da
4,ved næste folketingsvalg vil den fordeling vær...,da
7,fysiske egenskaber beskrives ofte som observer...,da
20,jæger/samlere bliver anset for at være vores a...,da
27,"ingen af dem blev dog valgt, ifølge class stru...",da
37,i forbindelse med udregning af støjspænding i ...,da
39,hvilket beskrives af plancks lov.,da
43,dvs.,da


In [8]:
len(test.loc[test["label"] == "da"])

6425